In [1]:
import pandas as pd
from bson import json_util 
import re

df = pd.read_csv("../data/eurocup_2020_results.csv")

In [3]:
events = df["events_list"][0]
pattern = r"{.*}"
eventos = re.findall(pattern, events[0])
len(eventos)

0

In [4]:
#Vamos a por datos de players

In [5]:
import requests
from bs4 import BeautifulSoup

n=1
url = f"https://www.fifaindex.com/es/players/fifa21_486/?page={n}"

In [6]:
res = requests.get(url)
res

<Response [200]>

In [7]:
soup = BeautifulSoup(res.text)


In [8]:
jugadores = soup.select("tr[data-playerid]")
len(jugadores)


30

In [9]:
jugadores[1]


<tr data-playerid="20801"><td><figure class="player"><a class="link-player" href="/es/player/20801/cristiano-ronaldo/fifa21/" title="Cristiano Ronaldo FIFA 21"><img alt="Cristiano Ronaldo FIFA 21" class="player size-5" loading="lazy" src="https://fifastatic.fifaindex.com/FIFA21/images/players/5/20801.png" srcset="https://fifastatic.fifaindex.com/FIFA21/images/players/5/20801@2x.png 2x, https://fifastatic.fifaindex.com/FIFA21/images/players/5/20801@3x.png 3x"/></a></figure></td><td data-title="Nacionalidad"><a class="link-nation" href="/es/players/fifa21/?nationality=38" title="Portugal"><img alt="Portugal" class="nation size-2" loading="lazy" src="https://fifastatic.fifaindex.com/FIFA21/images/flags/2/38.png" srcset="https://fifastatic.fifaindex.com/FIFA21/images/flags/2/38@2x.png 2x, https://fifastatic.fifaindex.com/FIFA21/images/flags/2/38@3x.png 3x"/></a></td><td data-title="MED / POT"><span class="badge badge-dark rating r1">92</span><span class="badge badge-dark rating r1">92</spa

In [10]:

#OBTENEMOS IMAGENES
def get_img(tag):
    pattern = r"src=\".*\" "
    link_img = re.findall(pattern,str(tag))[0]
    link_img = link_img.replace("src=","").replace('"',"").strip()
    return link_img 


In [11]:

#CLASE link-player
c_link_player = jugadores[1].select("a[class=link-player]")
#Nombre Jugador
print(c_link_player[1].text)
#imagen Jugador
print(get_img(c_link_player[0]))

Cristiano Ronaldo
https://fifastatic.fifaindex.com/FIFA21/images/players/5/20801.png


In [12]:
#CLASE link-nation
c_link_nation = jugadores[1].select("a[class=link-nation]")[0]
#Nombre PAIS
print(c_link_nation.get("title"))
#imagen Pais
print(get_img(c_link_nation))

Portugal
https://fifastatic.fifaindex.com/FIFA21/images/flags/2/38.png


In [13]:
#CLASE link-team
c_link_team = jugadores[1].select("a[class=link-team]")[0]
#Nombre Equipo
print(c_link_team.get("title").replace(" FIFA 21",""))
#Imagen Equipo
print(get_img(c_link_team))

Juventus de Turín
https://fifastatic.fifaindex.com/FIFA19/images/crest/3/light/45.png


In [14]:
#POSICIONES
pos = []
for e in jugadores[1].select("a[class=link-position]"):
    pos.append(e.text)
pos

['DC', 'EI']

In [15]:
def get_player_data(player):
    player_data = {}
    pos = []
    #OBTENEMOS IMAGENES
    def get_img(tag):
        pattern = r"src=\".*\" "
        link_img = re.findall(pattern,str(tag))[0]
        link_img = link_img.replace("src=","").replace('"',"").strip()
        return link_img 
    
    #a CLASE link-player
    c_link_player = player.select("a[class=link-player]")   
    #a CLASE link-nation
    c_link_nation = player.select("a[class=link-nation]")[0]
    for e in player.select("a[class=link-position]"):
        pos.append(e.text)
    #a CLASE link-team
    c_link_team = player.select("a[class=link-team]")[0]
    
    player_data ={
        "name": c_link_player[1].text,
        "p_photo": get_img(c_link_player[0]),
        "contry": c_link_nation.get("title"),
        "c_img": get_img(c_link_nation),
        "posiciones": pos,
        "club":c_link_team.get("title").replace(" FIFA 21",""),
        "club_img":get_img(c_link_team)
    }
                           
    return player_data

In [16]:
get_player_data(jugadores[1])

{'name': 'Cristiano Ronaldo',
 'p_photo': 'https://fifastatic.fifaindex.com/FIFA21/images/players/5/20801.png',
 'contry': 'Portugal',
 'c_img': 'https://fifastatic.fifaindex.com/FIFA21/images/flags/2/38.png',
 'posiciones': ['DC', 'EI'],
 'club': 'Juventus de Turín',
 'club_img': 'https://fifastatic.fifaindex.com/FIFA19/images/crest/3/light/45.png'}

In [17]:
#632 paginas
import json

tf = open("players_bd_eng.json", "w")

bd = []
for i in range(1,632):
    direc = f"https://www.fifaindex.com/players/fifa21_486/?page={i}"
    peticion = requests.get(direc)
    soup_page = BeautifulSoup(peticion.text)
    
    jug = soup_page.select("tr[data-playerid]")

    long = len(jug)
    #print(f"pagina {i}")
    #break
    for j in range(1,long+1):
        #print(j)
        #bd.append(get_player_data(jugadores[j]))
        json.dump(get_player_data(jug[j]),tf)
                  
tf.close()

KeyboardInterrupt: 

In [216]:
import json
my_dict={}
tf = open("myDictionary.json", "w")
json.dump(my_dict,tf)
tf.close()

In [118]:
direc = f"https://www.fifaindex.com/teams/fifa21_486/?page={1}"
peticion = requests.get(direc)
soup_page = BeautifulSoup(peticion.text)
equipos = soup_page.select("body > main > div > div > div> div> table > tbody > tr:nth-of-type(n+3)")

len(equipos[0])

#body > main > div > div > div.col-lg-8 > div.responsive-table.table-rounded > table > tbody

8

In [74]:
c_link_team = equipos[5].select("a[class=link-team]")
c_link_team[1].text

'Real Betis Balompié'

In [75]:
get_img(c_link_team[0])

'https://fifastatic.fifaindex.com/FIFA21/images/crest/5/light/449.png'

In [86]:
c_link_league = equipos[0].select("a[class=link-league]")[0]
c_link_league.text

'LaLiga Santander'

In [80]:
def get_team_data(team):
    player_data = {}
    pos = []   
    #a CLASE link-league
    c_link_team = team.select("a[class=link-team]") 
    #a CLASE link-team
    c_link_league = team.select("a[class=link-league]")[0]
    
    team_data ={
        "name": c_link_team[1].text,
        "t_flag": get_img(c_link_team[0]),
        "league":c_link_league.text
    }
    return team_data

In [115]:
#def get_all_players():
tf = open("teams_bd.json", "w")
bd = []
for i in range(1,24):
    direc = f"https://www.fifaindex.com/teams/fifa21_486/?page={i}"
    peticion = requests.get(direc)
    soup_page = BeautifulSoup(peticion.text)
    eqs = soup_page.select("body > main > div > div > div> div> table > tbody > tr:nth-of-type(n+3)")

    long = len(eqs)
    for j in range(1,long):
        if (len(eqs[j].select("td[colspan]")) > 0) or (len(eqs[j])== 0):
            print(f"Procesando pagina {i} de 23, equipo {j} NO es equipo")
        else:
            print(f"Procesando pagina {i} de 23, equipo {j}")
            json.dump(get_team_data(eqs[j]),tf)                  
tf.close()
    


Procesando pagina 1 de 23, equipo 1
Procesando pagina 1 de 23, equipo 2
Procesando pagina 1 de 23, equipo 3
Procesando pagina 1 de 23, equipo 4
Procesando pagina 1 de 23, equipo 5 NO es equipo
Procesando pagina 1 de 23, equipo 6 NO es equipo
Procesando pagina 1 de 23, equipo 7
Procesando pagina 1 de 23, equipo 8
Procesando pagina 1 de 23, equipo 9
Procesando pagina 1 de 23, equipo 10
Procesando pagina 1 de 23, equipo 11
Procesando pagina 1 de 23, equipo 12 NO es equipo
Procesando pagina 1 de 23, equipo 13 NO es equipo
Procesando pagina 1 de 23, equipo 14
Procesando pagina 1 de 23, equipo 15
Procesando pagina 1 de 23, equipo 16
Procesando pagina 1 de 23, equipo 17
Procesando pagina 1 de 23, equipo 18
Procesando pagina 1 de 23, equipo 19 NO es equipo
Procesando pagina 1 de 23, equipo 20 NO es equipo
Procesando pagina 1 de 23, equipo 21
Procesando pagina 1 de 23, equipo 22
Procesando pagina 1 de 23, equipo 23
Procesando pagina 1 de 23, equipo 24
Procesando pagina 1 de 23, equipo 25
Proces

Procesando pagina 7 de 23, equipo 1
Procesando pagina 7 de 23, equipo 2
Procesando pagina 7 de 23, equipo 3
Procesando pagina 7 de 23, equipo 4
Procesando pagina 7 de 23, equipo 5 NO es equipo
Procesando pagina 7 de 23, equipo 6 NO es equipo
Procesando pagina 7 de 23, equipo 7
Procesando pagina 7 de 23, equipo 8
Procesando pagina 7 de 23, equipo 9
Procesando pagina 7 de 23, equipo 10
Procesando pagina 7 de 23, equipo 11
Procesando pagina 7 de 23, equipo 12 NO es equipo
Procesando pagina 7 de 23, equipo 13 NO es equipo
Procesando pagina 7 de 23, equipo 14
Procesando pagina 7 de 23, equipo 15
Procesando pagina 7 de 23, equipo 16
Procesando pagina 7 de 23, equipo 17
Procesando pagina 7 de 23, equipo 18
Procesando pagina 7 de 23, equipo 19 NO es equipo
Procesando pagina 7 de 23, equipo 20 NO es equipo
Procesando pagina 7 de 23, equipo 21
Procesando pagina 7 de 23, equipo 22
Procesando pagina 7 de 23, equipo 23
Procesando pagina 7 de 23, equipo 24
Procesando pagina 7 de 23, equipo 25
Proces

Procesando pagina 14 de 23, equipo 1
Procesando pagina 14 de 23, equipo 2
Procesando pagina 14 de 23, equipo 3
Procesando pagina 14 de 23, equipo 4
Procesando pagina 14 de 23, equipo 5 NO es equipo
Procesando pagina 14 de 23, equipo 6 NO es equipo
Procesando pagina 14 de 23, equipo 7
Procesando pagina 14 de 23, equipo 8
Procesando pagina 14 de 23, equipo 9
Procesando pagina 14 de 23, equipo 10
Procesando pagina 14 de 23, equipo 11
Procesando pagina 14 de 23, equipo 12 NO es equipo
Procesando pagina 14 de 23, equipo 13 NO es equipo
Procesando pagina 14 de 23, equipo 14
Procesando pagina 14 de 23, equipo 15
Procesando pagina 14 de 23, equipo 16
Procesando pagina 14 de 23, equipo 17
Procesando pagina 14 de 23, equipo 18
Procesando pagina 14 de 23, equipo 19 NO es equipo
Procesando pagina 14 de 23, equipo 20 NO es equipo
Procesando pagina 14 de 23, equipo 21
Procesando pagina 14 de 23, equipo 22
Procesando pagina 14 de 23, equipo 23
Procesando pagina 14 de 23, equipo 24
Procesando pagina 1

Procesando pagina 20 de 23, equipo 1
Procesando pagina 20 de 23, equipo 2
Procesando pagina 20 de 23, equipo 3
Procesando pagina 20 de 23, equipo 4
Procesando pagina 20 de 23, equipo 5 NO es equipo
Procesando pagina 20 de 23, equipo 6 NO es equipo
Procesando pagina 20 de 23, equipo 7
Procesando pagina 20 de 23, equipo 8
Procesando pagina 20 de 23, equipo 9
Procesando pagina 20 de 23, equipo 10
Procesando pagina 20 de 23, equipo 11
Procesando pagina 20 de 23, equipo 12 NO es equipo
Procesando pagina 20 de 23, equipo 13 NO es equipo
Procesando pagina 20 de 23, equipo 14
Procesando pagina 20 de 23, equipo 15
Procesando pagina 20 de 23, equipo 16
Procesando pagina 20 de 23, equipo 17
Procesando pagina 20 de 23, equipo 18
Procesando pagina 20 de 23, equipo 19 NO es equipo
Procesando pagina 20 de 23, equipo 20 NO es equipo
Procesando pagina 20 de 23, equipo 21
Procesando pagina 20 de 23, equipo 22
Procesando pagina 20 de 23, equipo 23
Procesando pagina 20 de 23, equipo 24
Procesando pagina 2

In [119]:
def get_img(tag):
    pattern = r"src=\".*\" "
    link_img = re.findall(pattern,str(tag))[0]
    link_img = link_img.replace("src=","").replace('"',"").strip()
    return link_img 

#funcion que extrae la info de cada jugador
def get_player_data(player):
    player_data = {}
    pos = []   
    #a CLASE link-player
    c_link_player = player.select("a[class=link-player]")   
    #a CLASE link-nation
    c_link_nation = player.select("a[class=link-nation]")[0]
    for e in player.select("a[class=link-position]"):
        pos.append(e.text)
    #a CLASE link-team
    c_link_team = player.select("a[class=link-team]")[0]
    
    player_data ={
        "name": c_link_player[1].text,
        "p_photo": get_img(c_link_player[0]),
        "contry": c_link_nation.get("title"),
        "c_img": get_img(c_link_nation),
        "posiciones": pos,
        "club":c_link_team.get("title").replace(" FIFA 21",""),
        "club_img":get_img(c_link_team)
    }                           
    return player_data


#Recorremos todas las paginas de jugadores de fifaindex y obtenemos la informacion de todos ellos


In [125]:
#def get_all_players():
tf = open("../data/players_bd.json", "w")
bd = []
for i in range(1,633):
    direc = f"https://www.fifaindex.com/players/fifa21_486/?page={i}"
    peticion = requests.get(direc)
    soup_page = BeautifulSoup(peticion.text)

    jug = soup_page.select("tr[data-playerid]")

    long = len(jug)
    print(long)
    for j in range(1,long):
        print(f"Procesando pagina {i} de 632, jugador {j}")
        bd.append(get_player_data(jug[j]))
json.dump(bd,tf)                  
tf.close()

30
Procesando pagina 1 de 632, jugador 1
Procesando pagina 1 de 632, jugador 2
Procesando pagina 1 de 632, jugador 3
Procesando pagina 1 de 632, jugador 4
Procesando pagina 1 de 632, jugador 5
Procesando pagina 1 de 632, jugador 6
Procesando pagina 1 de 632, jugador 7
Procesando pagina 1 de 632, jugador 8
Procesando pagina 1 de 632, jugador 9
Procesando pagina 1 de 632, jugador 10
Procesando pagina 1 de 632, jugador 11
Procesando pagina 1 de 632, jugador 12
Procesando pagina 1 de 632, jugador 13
Procesando pagina 1 de 632, jugador 14
Procesando pagina 1 de 632, jugador 15
Procesando pagina 1 de 632, jugador 16
Procesando pagina 1 de 632, jugador 17
Procesando pagina 1 de 632, jugador 18
Procesando pagina 1 de 632, jugador 19
Procesando pagina 1 de 632, jugador 20
Procesando pagina 1 de 632, jugador 21
Procesando pagina 1 de 632, jugador 22
Procesando pagina 1 de 632, jugador 23
Procesando pagina 1 de 632, jugador 24
Procesando pagina 1 de 632, jugador 25
Procesando pagina 1 de 632, jug

30
Procesando pagina 9 de 632, jugador 1
Procesando pagina 9 de 632, jugador 2
Procesando pagina 9 de 632, jugador 3
Procesando pagina 9 de 632, jugador 4
Procesando pagina 9 de 632, jugador 5
Procesando pagina 9 de 632, jugador 6
Procesando pagina 9 de 632, jugador 7
Procesando pagina 9 de 632, jugador 8
Procesando pagina 9 de 632, jugador 9
Procesando pagina 9 de 632, jugador 10
Procesando pagina 9 de 632, jugador 11
Procesando pagina 9 de 632, jugador 12
Procesando pagina 9 de 632, jugador 13
Procesando pagina 9 de 632, jugador 14
Procesando pagina 9 de 632, jugador 15
Procesando pagina 9 de 632, jugador 16
Procesando pagina 9 de 632, jugador 17
Procesando pagina 9 de 632, jugador 18
Procesando pagina 9 de 632, jugador 19
Procesando pagina 9 de 632, jugador 20
Procesando pagina 9 de 632, jugador 21
Procesando pagina 9 de 632, jugador 22
Procesando pagina 9 de 632, jugador 23
Procesando pagina 9 de 632, jugador 24
Procesando pagina 9 de 632, jugador 25
Procesando pagina 9 de 632, jug

30
Procesando pagina 17 de 632, jugador 1
Procesando pagina 17 de 632, jugador 2
Procesando pagina 17 de 632, jugador 3
Procesando pagina 17 de 632, jugador 4
Procesando pagina 17 de 632, jugador 5
Procesando pagina 17 de 632, jugador 6
Procesando pagina 17 de 632, jugador 7
Procesando pagina 17 de 632, jugador 8
Procesando pagina 17 de 632, jugador 9
Procesando pagina 17 de 632, jugador 10
Procesando pagina 17 de 632, jugador 11
Procesando pagina 17 de 632, jugador 12
Procesando pagina 17 de 632, jugador 13
Procesando pagina 17 de 632, jugador 14
Procesando pagina 17 de 632, jugador 15
Procesando pagina 17 de 632, jugador 16
Procesando pagina 17 de 632, jugador 17
Procesando pagina 17 de 632, jugador 18
Procesando pagina 17 de 632, jugador 19
Procesando pagina 17 de 632, jugador 20
Procesando pagina 17 de 632, jugador 21
Procesando pagina 17 de 632, jugador 22
Procesando pagina 17 de 632, jugador 23
Procesando pagina 17 de 632, jugador 24
Procesando pagina 17 de 632, jugador 25
Proces

30
Procesando pagina 25 de 632, jugador 1
Procesando pagina 25 de 632, jugador 2
Procesando pagina 25 de 632, jugador 3
Procesando pagina 25 de 632, jugador 4
Procesando pagina 25 de 632, jugador 5
Procesando pagina 25 de 632, jugador 6
Procesando pagina 25 de 632, jugador 7
Procesando pagina 25 de 632, jugador 8
Procesando pagina 25 de 632, jugador 9
Procesando pagina 25 de 632, jugador 10
Procesando pagina 25 de 632, jugador 11
Procesando pagina 25 de 632, jugador 12
Procesando pagina 25 de 632, jugador 13
Procesando pagina 25 de 632, jugador 14
Procesando pagina 25 de 632, jugador 15
Procesando pagina 25 de 632, jugador 16
Procesando pagina 25 de 632, jugador 17
Procesando pagina 25 de 632, jugador 18
Procesando pagina 25 de 632, jugador 19
Procesando pagina 25 de 632, jugador 20
Procesando pagina 25 de 632, jugador 21
Procesando pagina 25 de 632, jugador 22
Procesando pagina 25 de 632, jugador 23
Procesando pagina 25 de 632, jugador 24
Procesando pagina 25 de 632, jugador 25
Proces

30
Procesando pagina 33 de 632, jugador 1
Procesando pagina 33 de 632, jugador 2
Procesando pagina 33 de 632, jugador 3
Procesando pagina 33 de 632, jugador 4
Procesando pagina 33 de 632, jugador 5
Procesando pagina 33 de 632, jugador 6
Procesando pagina 33 de 632, jugador 7
Procesando pagina 33 de 632, jugador 8
Procesando pagina 33 de 632, jugador 9
Procesando pagina 33 de 632, jugador 10
Procesando pagina 33 de 632, jugador 11
Procesando pagina 33 de 632, jugador 12
Procesando pagina 33 de 632, jugador 13
Procesando pagina 33 de 632, jugador 14
Procesando pagina 33 de 632, jugador 15
Procesando pagina 33 de 632, jugador 16
Procesando pagina 33 de 632, jugador 17
Procesando pagina 33 de 632, jugador 18
Procesando pagina 33 de 632, jugador 19
Procesando pagina 33 de 632, jugador 20
Procesando pagina 33 de 632, jugador 21
Procesando pagina 33 de 632, jugador 22
Procesando pagina 33 de 632, jugador 23
Procesando pagina 33 de 632, jugador 24
Procesando pagina 33 de 632, jugador 25
Proces

30
Procesando pagina 41 de 632, jugador 1
Procesando pagina 41 de 632, jugador 2
Procesando pagina 41 de 632, jugador 3
Procesando pagina 41 de 632, jugador 4
Procesando pagina 41 de 632, jugador 5
Procesando pagina 41 de 632, jugador 6
Procesando pagina 41 de 632, jugador 7
Procesando pagina 41 de 632, jugador 8
Procesando pagina 41 de 632, jugador 9
Procesando pagina 41 de 632, jugador 10
Procesando pagina 41 de 632, jugador 11
Procesando pagina 41 de 632, jugador 12
Procesando pagina 41 de 632, jugador 13
Procesando pagina 41 de 632, jugador 14
Procesando pagina 41 de 632, jugador 15
Procesando pagina 41 de 632, jugador 16
Procesando pagina 41 de 632, jugador 17
Procesando pagina 41 de 632, jugador 18
Procesando pagina 41 de 632, jugador 19
Procesando pagina 41 de 632, jugador 20
Procesando pagina 41 de 632, jugador 21
Procesando pagina 41 de 632, jugador 22
Procesando pagina 41 de 632, jugador 23
Procesando pagina 41 de 632, jugador 24
Procesando pagina 41 de 632, jugador 25
Proces

30
Procesando pagina 49 de 632, jugador 1
Procesando pagina 49 de 632, jugador 2
Procesando pagina 49 de 632, jugador 3
Procesando pagina 49 de 632, jugador 4
Procesando pagina 49 de 632, jugador 5
Procesando pagina 49 de 632, jugador 6
Procesando pagina 49 de 632, jugador 7
Procesando pagina 49 de 632, jugador 8
Procesando pagina 49 de 632, jugador 9
Procesando pagina 49 de 632, jugador 10
Procesando pagina 49 de 632, jugador 11
Procesando pagina 49 de 632, jugador 12
Procesando pagina 49 de 632, jugador 13
Procesando pagina 49 de 632, jugador 14
Procesando pagina 49 de 632, jugador 15
Procesando pagina 49 de 632, jugador 16
Procesando pagina 49 de 632, jugador 17
Procesando pagina 49 de 632, jugador 18
Procesando pagina 49 de 632, jugador 19
Procesando pagina 49 de 632, jugador 20
Procesando pagina 49 de 632, jugador 21
Procesando pagina 49 de 632, jugador 22
Procesando pagina 49 de 632, jugador 23
Procesando pagina 49 de 632, jugador 24
Procesando pagina 49 de 632, jugador 25
Proces

30
Procesando pagina 57 de 632, jugador 1
Procesando pagina 57 de 632, jugador 2
Procesando pagina 57 de 632, jugador 3
Procesando pagina 57 de 632, jugador 4
Procesando pagina 57 de 632, jugador 5
Procesando pagina 57 de 632, jugador 6
Procesando pagina 57 de 632, jugador 7
Procesando pagina 57 de 632, jugador 8
Procesando pagina 57 de 632, jugador 9
Procesando pagina 57 de 632, jugador 10
Procesando pagina 57 de 632, jugador 11
Procesando pagina 57 de 632, jugador 12
Procesando pagina 57 de 632, jugador 13
Procesando pagina 57 de 632, jugador 14
Procesando pagina 57 de 632, jugador 15
Procesando pagina 57 de 632, jugador 16
Procesando pagina 57 de 632, jugador 17
Procesando pagina 57 de 632, jugador 18
Procesando pagina 57 de 632, jugador 19
Procesando pagina 57 de 632, jugador 20
Procesando pagina 57 de 632, jugador 21
Procesando pagina 57 de 632, jugador 22
Procesando pagina 57 de 632, jugador 23
Procesando pagina 57 de 632, jugador 24
Procesando pagina 57 de 632, jugador 25
Proces

30
Procesando pagina 65 de 632, jugador 1
Procesando pagina 65 de 632, jugador 2
Procesando pagina 65 de 632, jugador 3
Procesando pagina 65 de 632, jugador 4
Procesando pagina 65 de 632, jugador 5
Procesando pagina 65 de 632, jugador 6
Procesando pagina 65 de 632, jugador 7
Procesando pagina 65 de 632, jugador 8
Procesando pagina 65 de 632, jugador 9
Procesando pagina 65 de 632, jugador 10
Procesando pagina 65 de 632, jugador 11
Procesando pagina 65 de 632, jugador 12
Procesando pagina 65 de 632, jugador 13
Procesando pagina 65 de 632, jugador 14
Procesando pagina 65 de 632, jugador 15
Procesando pagina 65 de 632, jugador 16
Procesando pagina 65 de 632, jugador 17
Procesando pagina 65 de 632, jugador 18
Procesando pagina 65 de 632, jugador 19
Procesando pagina 65 de 632, jugador 20
Procesando pagina 65 de 632, jugador 21
Procesando pagina 65 de 632, jugador 22
Procesando pagina 65 de 632, jugador 23
Procesando pagina 65 de 632, jugador 24
Procesando pagina 65 de 632, jugador 25
Proces

KeyboardInterrupt: 